<a href="https://colab.research.google.com/github/lcnature/statsthinking21-python/blob/master/notebooks/14-ComparingMeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.power import TTestIndPower, TTestPower
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.weightstats import ttest_ind
from statsmodels.stats import weightstats as stests
import matplotlib.pyplot as plt
import seaborn as sns
! pip install nhanes
from nhanes.load import load_NHANES_data
! pip install pingouin

nhanes_data = load_NHANES_data()
adult_nhanes_data = nhanes_data.query('AgeInYearsAtScreening > 17')

## Chapter 14: Comparing Means in Python

### 14.1 Testing the value of a single mean (Section 14.1)

In this example, we will show multiple ways to test a hypothesis about the value of a single mean. As an example, let’s test whether the mean systolic blood pressure (BP) in the NHANES dataset (averaged over the three measurements that were taken for each person) is greater than 120 mm, which is the standard value for normal systolic BP.

First let’s perform a power analysis to see how large our sample would need to be in order to detect a small difference (Cohen’s d = .2).

In [ ]:
from statsmodels.stats.power import TTestPower

# Conduct power analysis for one-sample t-test
effect_size = 0.2
power = 0.8
alpha = 0.05

analysis = TTestPower()
sample_size = analysis.solve_power(effect_size=effect_size, power=power, alpha=alpha, alternative='larger')
print("Required sample size: ", sample_size)

Based on this, we take a sample of 156 individuals from the dataset.


In [ ]:
n = int(np.ceil(sample_size))
adult_nhanes_data.loc[:,'BPSysAve'] = adult_nhanes_data.loc[:,['SystolicBloodPres1StRdgMmHg',
                                                   'SystolicBloodPres2NdRdgMmHg',
                                                   'SystolicBloodPres3RdRdgMmHg']].mean(axis=1)

nhanes_BP_sample = adult_nhanes_data.dropna(subset=['BPSysAve']).sample(n=n)
mean_bp = nhanes_BP_sample['BPSysAve'].mean()
print("Mean BP: ",mean_bp)

Let’s perform a one-sample t-test:

In [ ]:
t_stat, p_value = stats.ttest_1samp(nhanes_BP_sample['BPSysAve'], popmean=120)
print("One-sample t-test:")
print("t-stat =", t_stat)
print("p-value =",p_value)

Surprisingly, we actually got a statistically significant effect,meaning that the average systolic blood pressure is higher than the population mean in this sample.


We might also want to quantify the degree of evidence in favor of the null hypothesis, which we can do using the Bayes Factor:


In [ ]:
import pingouin as pg

bf = pg.bayesfactor_ttest(t=t_stat, nx=n, paired=False)
print(f"Bayes Factor: {bf}")

The Bayes factor being much bigger than 1 suggests there is a strong evidence for the sample mean systolic blood pressure to be different from the population mean.

### 14.2 Comparing two means (Section 14.2)

To compare two means from independent samples, we can use the two-sample t-test. Let’s say that we want to compare blood pressure of smokers and non-smokers; we don’t have an expectation for the direction, so we will use a two-sided test. First let’s perform a power analysis, again for a small effect:


In [ ]:
from statsmodels.stats.power import TTestIndPower

effect_size = 0.2
analysis = TTestIndPower()
sample_size = analysis.solve_power(effect_size=effect_size, power=0.8, alpha=0.05, alternative='two-sided')
print("Required sample size per group: ",sample_size)

This tells us that we need 394 subjects in each group, so let’s sample 394 smokers and 394 nonsmokers from the NHANES dataset, and then put them into a single data frame with a variable denoting their smoking status.

In [ ]:
n = int(np.ceil(sample_size))
# clean up data
adult_nhanes_data.loc[adult_nhanes_data['SmokedAtLeast100CigarettesInLife'] == 0, 'DoYouNowSmokeCigarettes'] = 'Not at all'
adult_nhanes_data.loc[:, 'SmokeNow'] = (adult_nhanes_data['DoYouNowSmokeCigarettes'] != 'Not at all')

smokers = adult_nhanes_data[adult_nhanes_data['SmokeNow'] == 1].dropna(subset=['BPSysAve']).sample(n=n)
nonsmokers = adult_nhanes_data[adult_nhanes_data['SmokeNow'] == 0].dropna(subset=['BPSysAve']).sample(n=n)




Let’s test our hypothesis using a standard two-sample t-test.

In [ ]:
t_stat, p_value = stats.ttest_ind(smokers['BPSysAve'], nonsmokers['BPSysAve'], equal_var=False)
print("Two-sample t-test: ")
print("t-stat =",t_stat)
print("p-value =",p_value)

This shows us that there is not a significant difference of blood pressure between smokers and non-smokers.

Let’s look at the Bayes factor to quantify the evidence:

In [ ]:
bf_two_sample = pg.bayesfactor_ttest(t_stat, nx=n, ny=n)
print("Bayes Factor: ", bf_two_sample)

Because `pg.bayesfactor_ttest` provides Bayes factor in favor of the alternative hypothesis and this number is quite small. It suggests there is positive evidence in favor of the null hypothesis.

### 14.3 The t-test as a linear model (Section 14.3)

We can also use linear regression to implement these t-tests.

The one-sample t-test is basically a test for whether the intercept is different from zero, so we use a model with only an intercept and apply this to the data after subtracting the null hypothesis mean (so that the expectation under the null hypothesis is an intercept of zero):

In [ ]:
nhanes_BP_sample['BPSysAveDiff'] = nhanes_BP_sample['BPSysAve'] - 120
regression_result_1samp = ols('BPSysAveDiff ~ 1', data=nhanes_BP_sample).fit()
print(regression_result_1samp.summary())

In [ ]:
print('p-value:', regression_result_1samp.pvalues[0])

You should notice that the p-value is the same as that from the `scipy.stats.ttest_1samp`

We can also run the two-sample t-test using `ols`:

In [ ]:
# combine the smokers' and non-smokers' samples into a single column
sample_df = pd.concat([smokers, nonsmokers])

regression_result_2samp = ols('BPSysAve ~ SmokeNow', data=sample_df).fit()
print(regression_result_2samp.summary())

Likewise, you will find that the p-value for the coefficient associated with `SmokeNow` is the same as the p-value returned by `scipy.stats.ttest_ind`.

### 14.4 Comparing paired observations (Section 14.4)

In [ ]:
cleaned_data = adult_nhanes_data.dropna(subset=['SystolicBloodPres1StRdgMmHg',
                                                'SystolicBloodPres2NdRdgMmHg'])

t_stat, p_value = stats.ttest_rel(cleaned_data['SystolicBloodPres1StRdgMmHg'],
                                  cleaned_data['SystolicBloodPres2NdRdgMmHg'])
print("Paired t-test: ")
print("t-stat =",t_stat)
print("p-value =",p_value)

In fact, paired t-test is equivalent to one-sample t-test on the difference between the scores across all individuals. You will find the code below generates the same t-score and p-value:


In [ ]:
t_stat, p_value = stats.ttest_1samp(
    cleaned_data['SystolicBloodPres1StRdgMmHg']
    - cleaned_data['SystolicBloodPres2NdRdgMmHg'], popmean=0)
print("Paired t-test performed using one-sample t-test: ")
print("t-stat =",t_stat)
print("p-value =",p_value)